# PyCity Schools Analysis

- Overall, Charter Schools that have smaller class sizes and less funding produce better test results than District Schools with more students and higher budgets. Spending more money per student does not appear to have a positive impact on improving test scores. 

- Charter schools appear to be better able to achieve higher test scores with less money to spend per student. The top 5 schools are all charter schools and they are all able to maintain Math and Reading scores at an average of 83 points across all grades, with a great passing rate of 90.43% overall.

- District schools have much larger class sizes, above 2,000 students, as well as a higher budget per student, but that does not translate into higher test scores. More students are struggling with math than with reading but, overall, only 53.67% of District School students were able to pass both math and reading. 

- While it’s clear that Charter schools out perform District schools, it is not readily apparent as to why that is. From our available data, we can infer that class size might be an important factor, but it might be beneficial to expand our analysis to look into the in-school resources made available to students (such as tutoring and library services) as well as an overview of the teachers (eg: skill level, years teaching, salaries, job satisfaction).

- It would also be helpful to do an additional analysis to look at the demographics of the schools and the students. The needs of the individual schools will vary depending on the individual students that make up the student body. A quick snapshot of these particulars might give us a better understanding of why certain schools might be failing or succeeding. These reports, as well as a breakdown of exactly what each school spends its budget on,  would be very useful in helping us achieve our goal of making strategic decisions regarding future school budgets and priorities.  

In [ ]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load 
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

school_data_complete.head()

# District Summary

In [ ]:
# Calculate the total number of unique schools
school_count = school_data_complete["school_name"].nunique()
school_count

In [ ]:
# Calculate the total number of students
student_count = school_data_complete["student_name"].count()
student_count

In [ ]:
# Calculate the total budget
total_budget = school_data_complete["budget"].unique().sum()
total_budget

In [ ]:
# Calculate the average math score
average_math_score = school_data_complete["math_score"].mean()
average_math_score

In [ ]:
# Calculate the average reading score
average_reading_score = school_data_complete["reading_score"].mean()
average_reading_score

In [ ]:
# Calculate the percentage of students who passed math with a score greather than or equal to 70
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count /(student_count) * 100
passing_math_percentage

In [ ]:
# Calculate the percentage of students who passeed reading with a score greather than or equal to 70
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_count = passing_reading_count /(student_count) * 100
passing_reading_count

In [ ]:
# Calculate the percentage of students that passed both math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count / (student_count) * 100
overall_passing_rate

In [ ]:
# Assemble into high-level DataFrame with key metrics called 'district_summary'
district_summary = pd.DataFrame({"Total Schools":[school_count], 
                     "Total Students":[student_count], 
                     "Total Budget":[total_budget], 
                     "Average Math Score":[average_math_score], 
                     "Average Reading Score":[average_reading_score], 
                     "% Passing Math":[passing_math_percentage], 
                     "% Passing Reading":[passing_reading_count], 
                     "% Overall Passing":[overall_passing_rate]})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)


# Display the DataFrame
district_summary

# School Summary

In [ ]:
# Select the school type
school_types = school_data.set_index(["school_name"])["type"]

In [ ]:
# Calculate the total student count
per_school_counts = school_data_complete["school_name"].value_counts()

In [ ]:
# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
per_school_capita = per_school_budget / per_school_counts

In [ ]:
# Calculate the average test scores
per_school_math = school_data_complete.groupby(["school_name"]).mean()["math_score"]
per_school_reading = school_data_complete.groupby(["school_name"]).mean()["reading_score"]

In [ ]:
# Calculate the number of schools with math scores of 70 or higher
school_passing_math = school_data_complete[school_data_complete["math_score"] >= 70]

In [ ]:
# Calculate the number of schools with reading scores of 70 or higher
school_passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70]

In [ ]:
# Calculate the schools that passed both math and reading with scores of 70 or higher
passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)]

In [ ]:
# Calculate the passing rates
per_school_passing_math = school_passing_math.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
per_school_passing_reading = school_passing_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
overall_passing_rate = passing_math_and_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100

In [ ]:
# Assemble into DataFrame called 'per_school_summary'
per_school_summary = pd.DataFrame({
             "School Type": school_types,
             "Total Students": per_school_counts,
             "Total School Budget": per_school_budget,
             "Per Student Budget": per_school_capita,
             "Average Math Score": per_school_math,
           "Average Reading Score": per_school_reading,
           "% Passing Math": per_school_passing_math,
           "% Passing Reading": per_school_passing_reading,
           "% Overall Passing": overall_passing_rate})


# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

# Highest-Performing Schools by % of Overall Passing

In [ ]:
# Sort the schools by '% Overall Passing' in descending order
top_schools = per_school_summary.sort_values(["% Overall Passing"], ascending=False)

In [ ]:
# Display the top 5 rows
top_schools.head(5)

# Lowest-Performing Schools by % of Overall Passing

In [ ]:
# Sort the schools by '% Overall Passing' in ascending order
bottom_schools = per_school_summary.sort_values(["% Overall Passing"], ascending=True)

In [ ]:
# Display the top 5 rows
bottom_schools.head(5)

# Math Scores by Grade

In [ ]:
# Separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

In [ ]:
# Group by 'school_name' and take the mean of each
ninth_graders_scores = ninth_graders.groupby("school_name").mean()
tenth_graders_scores = tenth_graders.groupby("school_name").mean()
eleventh_graders_scores = eleventh_graders.groupby("school_name").mean()
twelfth_graders_scores = twelfth_graders.groupby("school_name").mean()

In [ ]:
# Use the code to select only the 'math_score'
ninth_grade_math_scores = ninth_graders_scores["math_score"]
tenth_grade_math_scores = tenth_graders_scores["math_score"]
eleventh_grade_math_scores = eleventh_graders_scores["math_score"]
twelfth_grade_math_scores = twelfth_graders_scores["math_score"]

In [ ]:
# Assemble into DataFrame called 'math_scores_by_grade'
math_scores_by_grade = pd.DataFrame({ "9th" : ninth_grade_math_scores,
                                     "10th" : tenth_grade_math_scores,
                                     "11th" : eleventh_grade_math_scores,
                                     "12th" : twelfth_grade_math_scores })
                        
# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

# Reading Scores by Grade

In [ ]:
# Separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

In [ ]:
# Group by 'school_name' and take the mean of each
ninth_graders_scores = ninth_graders.groupby("school_name").mean()
tenth_graders_scores = tenth_graders.groupby("school_name").mean()
eleventh_graders_scores = eleventh_graders.groupby("school_name").mean()
twelfth_graders_scores = twelfth_graders.groupby("school_name").mean()

In [ ]:
# Use the code to select only the 'reading_score'
ninth_grade_reading_scores = ninth_graders_scores["reading_score"]
tenth_grade_reading_scores = tenth_graders_scores["reading_score"]
eleventh_grade_reading_scores = eleventh_graders_scores["reading_score"]
twelfth_grade_reading_scores = twelfth_graders_scores["reading_score"]

In [ ]:
# Assemble into DataFrame called 'reading_scores_by_grade'
reading_scores_by_grade = pd.DataFrame({ "9th" : ninth_grade_reading_scores,
                                     "10th" : tenth_grade_reading_scores,
                                     "11th" : eleventh_grade_reading_scores,
                                     "12th" : twelfth_grade_reading_scores })
                        

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

# Scores by School Spending

In [ ]:
# Establish the bins 
bins = [0, 585, 630, 645, 680]
spending_labels = ["<$585", "$585-630", "$630-645", "$645-680"]

# Create a copy of the school summary for "Per Student Budget" 
school_spending_df = per_school_summary.copy()

# Use 'pd.cut' to categorize spending based on the bins
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, 
                                                             bins, labels=spending_labels)

In [ ]:
#  Calculate averages for the desired columns
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

In [ ]:
# Assemble into DataFrame called 'spending_summary'
spending_summary = pd.DataFrame({"Average Math Score" : spending_math_scores,
                                "Average Reading Score" : spending_reading_scores,
                                "% Passing Math" : spending_passing_math,
                                "% Passing Reading" : spending_passing_reading,
                                "% Passing Overall" : overall_passing_spending})

# Display the DataFrame
spending_summary

# Scores by School Size

In [ ]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.
per_school_summary["School Size"] = pd.cut(per_school_counts, size_bins, labels=labels)

In [ ]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary.groupby(["School Size"]).mean()["% Overall Passing"]

In [ ]:
# Assemble into DataFrame called 'size_summary'
size_summary = pd.DataFrame({"Average Math Score" : size_math_scores,
                             "Average Reading Score" : size_reading_scores,
                             "% Passing Math" : size_passing_math,
                             "% Passing Reading" : size_passing_reading,
                             "% Overall Passing" : size_overall_passing})

# Display the DataFrame
size_summary

# Scores by School Type

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = per_school_summary.groupby(["School Type"]).mean()
type_reading_scores = per_school_summary.groupby(["School Type"]).mean()
type_passing_math = per_school_summary.groupby(["School Type"]).mean()
type_passing_reading = per_school_summary.groupby(["School Type"]).mean()
type_overall_passing = per_school_summary.groupby(["School Type"]).mean()

In [ ]:
# Select new column data
average_math_score_by_type = type_math_scores["Average Math Score"]
average_reading_score_by_type = type_reading_scores["Average Reading Score"]
average_percent_passing_math_by_type = type_passing_math["% Passing Math"]
average_percent_passing_reading_by_type = type_passing_reading["% Passing Reading"]
average_percent_overall_passing_by_type = type_overall_passing["% Overall Passing"]

In [ ]:
# Assemble into DataFrame called 'type_summary'
type_summary = pd.DataFrame({"Average Math Score" : average_math_score_by_type,
                             "Average Reading Score" : average_reading_score_by_type,
                             "% Passing Math" : average_percent_passing_math_by_type,
                             "% Passing Reading" : average_percent_passing_reading_by_type,
                             "% Overall Passing" : average_percent_overall_passing_by_type})

# Display the DataFrame
type_summary